In [ ]:
import pandas as pd
import unicodedata
import re
from pymongo import MongoClient


In [50]:
#leitura do dataset

dataset = pd.read_csv('acidentes2024.csv', encoding='ISO-8859-1', delimiter=';')

print(dataset.head())

print("quantidade amostras", dataset.shape[0])

         id    pesid data_inversa     dia_semana   horario  uf   br   km  \
0  571789.0  1269011   2024-01-01  segunda-feira  03:56:00  ES  101   38   
1  571804.0  1269118   2024-01-01  segunda-feira  04:50:00  PI  343  185   
2  571806.0  1269735   2024-01-01  segunda-feira  04:30:00  BA  116  578   
3  571818.0  1269075   2024-01-01  segunda-feira  06:30:00  SE  101   18   
4  571838.0  1269159   2024-01-01  segunda-feira  05:00:00  MT  364  240   

            municipio                                     causa_acidente  ...  \
0  CONCEICAO DA BARRA                             Ultrapassagem Indevida  ...   
1            PIRIPIRI                        Manobra de mudança de faixa  ...   
2             BREJOES                   Ingestão de álcool pelo condutor  ...   
3    MALHADA DOS BOIS           Reação tardia ou ineficiente do condutor  ...   
4        RONDONOPOLIS  Condutor deixou de manter distância do veículo...  ...   

        sexo ilesos feridos_leves feridos_graves mortos 

In [51]:
# Função para separar 'marca' e 'modelo'
def split_marca_modelo(value):
    if isinstance(value, str) and '/' in value:
        return value.split('/', 1)  # Retorna uma lista com duas partes
    return [value, None]  # Retorna a marca e None para modelo se não houver barra

# Aplicar a função e criar novas colunas
dataset[['marca', 'modelo']] = dataset['marca'].apply(split_marca_modelo).apply(pd.Series)

In [53]:
# limpeza do dataset
dataset.replace("Não Informado", pd.NA, inplace=True)
dataset.replace("Ignorado", pd.NA, inplace=True)

dataset = dataset[dataset['tipo_veiculo'] == 'Automóvel']
dataset = dataset[dataset['ano_fabricacao_veiculo'] != 0]
dataset = dataset[dataset['idade'] < 100]

dataset = dataset.dropna()
print("quantidade amostras", dataset.shape[0])

quantidade amostras 46799


In [54]:
# normalização das strings

def normalize_string(s):
    # Remove acentos
    s = unicodedata.normalize('NFKD', s).encode('ASCII', 'ignore').decode('utf-8')
    # Transforma em minúsculas
    s = s.lower()
    # Substitui espaços e caracteres especiais por underscores
    s = re.sub(r'[^a-z0-9]+', '_', s)
    # Remove underscores do início e do fim
    s = s.strip('_')
    return s


col_to_normalize = ["dia_semana", "municipio", "causa_acidente", "tipo_acidente", "classificacao_acidente", "fase_dia", "sentido_via", "condicao_metereologica", "tipo_pista", "tracado_via", "uso_solo", "tipo_veiculo", "tipo_envolvido", "estado_fisico", "sexo" ]

for coluna in col_to_normalize:
    if coluna in dataset.columns:
        dataset[coluna] = dataset[coluna].apply(normalize_string)

In [55]:
#campos únicos
for col in dataset.columns:
    print(f"\nValores únicos na coluna '{col}':")
    print(dataset[col].unique())


Valores únicos na coluna 'id':
[571774. 571779. 571782. ... 635880. 635887. 636019.]

Valores únicos na coluna 'pesid':
[1268985 1268998 1268999 ... 1426084 1426083 1426372]

Valores únicos na coluna 'data_inversa':
['2024-01-01' '2024-01-02' '2024-01-03' '2024-01-04' '2024-01-05'
 '2024-01-06' '2024-01-07' '2024-01-08' '2024-01-09' '2024-01-10'
 '2024-01-11' '2024-01-12' '2024-01-13' '2024-01-14' '2024-01-15'
 '2024-01-16' '2024-01-17' '2024-01-18' '2024-01-19' '2024-01-20'
 '2024-01-21' '2024-01-22' '2024-01-23' '2024-01-24' '2024-01-25'
 '2024-01-26' '2024-01-27' '2024-01-28' '2024-01-29' '2024-01-30'
 '2024-01-31' '2024-02-01' '2024-02-02' '2024-02-03' '2024-02-04'
 '2024-02-05' '2024-02-06' '2024-02-07' '2024-02-08' '2024-02-09'
 '2024-02-10' '2024-02-11' '2024-02-12' '2024-02-13' '2024-02-14'
 '2024-02-15' '2024-02-16' '2024-02-17' '2024-02-18' '2024-02-19'
 '2024-02-20' '2024-02-21' '2024-02-22' '2024-02-23' '2024-02-24'
 '2024-02-25' '2024-02-26' '2024-02-27' '2024-02-28' '202

In [58]:
# inserir dataset no mongo

client = MongoClient('mongodb://localhost:27017/')
db = client['TCCMLRisk']
collection = db['vehicles-dataset']


# for index, row in dataset.iterrows():
#     document = row.to_dict()
#     collection.insert_one(document)

documents = dataset.to_dict(orient='records')
if documents:
    collection.insert_many(documents)

print(f"{len(documents)} documentos inseridos com sucesso!")


46799 documentos inseridos com sucesso!
